In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import os
import matplotlib.pyplot as plt
import json
from symspellpy import SymSpell
import contractions
import pickle

import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from tqdm import tqdm
import time
from PIL import Image


In [ ]:
# params
top_k = 5000 # nombre de mots à garder
embedding_dim = 300 # dimension de l'embedding
glove_dim = 300
units = 64 # nombre d'unités GRU et le nombre d'unités pour l'attention de chaque layer

BUFFER_SIZE = 1000
BATCH_SIZE = 256


split = .4 # 4% des données pour le train

# Data Import

In [ ]:
os.path.abspath('.')
base_path = "/home/cesi/datascience/corbeille"

In [ ]:
train_captions = os.path.abspath('.')+"/phase_6_7/annotations/captions_train2014.json"
train_captions = base_path+"/annotations/captions_train2014.json"

# test_captions = os.path.abspath('.')+"/phase_6_7/annotations/captions_val2014.json"

with open(train_captions, 'r') as f:
    annotations = json.load(f)
# with open(test_captions, 'r') as f:
    # test_annotations = json.load(f)

df = pd.DataFrame.from_dict(annotations['annotations'])
# test_df = pd.DataFrame.from_dict(test_annotations['annotations'])

# df['image_path'] = df.apply(lambda row: os.path.abspath('.')+"/phase_6_7/train2014/COCO_train2014_" + '%012d.jpg' % (row['image_id']), axis=1)
df['image_path'] = df.apply(lambda row: base_path+"/train2014/COCO_train2014_" + '%012d.jpg' % (row['image_id']), axis=1)
# test_df['image_path'] = test_df.apply(lambda row: os.path.abspath('.')+"/phase_6_7/val2014/COCO_val2014_" + '%012d.jpg' % (row['image_id']), axis=1)

# split into train and validation

train_df = df[:int(split*len(df))]
# train_df.sample(frac=1)

val_df = df[int((split)*len(df)):]
print(train_df.count())
#9s

In [ ]:
df

In [ ]:
#print df index 1
print(train_df)
train_df.iloc[1]['image_path']
img = plt.imread(train_df.iloc[1]['image_path'])
plt.imshow(img)

# Prep Data Annotations

In [ ]:
# Retrait des mots ayant moins que deux caractères
def remove_small(sentence):

    words = sentence.split() 
    words = [word for word in words if len(word)>2]
    return ' '.join(words)


mots_vides=stopwords.words('english')
def remove_stopwords(sentence):
    
    words = sentence.split()
    words = [word for word in words if word not in mots_vides]
    
    return ' '.join(words)

ponctuations = string.punctuation
def remove_ponctuation(sentence):
    
    # print(ponctuations)
    
    for char in sentence:
        if char in ponctuations:
            sentence = sentence.replace(char, '')
      
    return sentence

def nlp_pipeline(text):
    # Convertir les lettres majuscules en minuscules
    text = text.lower()
    
    # Remplacer la nouvelle ligne par un espace
    text = text.replace('\n', ' ').replace('\r', '')
    text = ' '.join(text.split())
    
    # Retirer les lettres majuscules, toutes les chaine de caractères qui ne sont pas des lettres ou des chiffres
    #À COMPLÉTER
    
    # Retirer les caractères spéciaux
    text = re.sub(r"(\s\-\s|-$)", "", text)       
    #À COMPLÉTER
    text = re.sub(r"\x89û", "", text)
    return text

def add_start_end_seq_token(caption):
    caption = f"<start> {caption} <end>" 
    return caption

# def remove_too_long_text(text):
#     return len(text.split()) < 30

def expand_contractions(text):
    return contractions.fix(text)

symsp = SymSpell()
symsp.load_dictionary('/home/cesi/datascience/data/frequency_dictionary.txt', term_index=0, count_index=1, separator=' ')

def correct_spelling(text):
    suggestions = symsp.lookup_compound(text, max_edit_distance=2)
    return suggestions[0].term

def clean_text(data):
    # data['text'] = data['text'].apply(lambda x : remove_url(x))
    # data['text'] = data['text'].apply(lambda x : remove_html(x))
    # data['text'] = data['text'].apply(lambda x : stem_words(x))
    # data['caption'] = data['caption'].apply(lambda x : remove_ponctuation(x))
    # data['caption'] = data['caption'].apply(lambda x : remove_stopwords(x))
    # data['text'] = data['text'].apply(lambda x : remove_emoji(x))
    # data['caption'] = data['caption'].apply(lambda x : remove_small(x))
    # data['text'] = data['text'].apply(lambda x : lem_word(x))
    data['caption'] = data['caption'].apply(lambda x : nlp_pipeline(x))
    data['caption'] = data['caption'].apply(lambda x : correct_spelling(x))
    data['caption'] = data['caption'].apply(lambda x : expand_contractions(x))
    data['caption'] = data['caption'].apply(lambda x : add_start_end_seq_token(x))

    return data

train_df = clean_text(train_df)

#20s

### Tokeniser && Data Annotation

In [ ]:
def get_tokenizer(df):
    def calc_max_length(tensor):
        return max(len(t) for t in tensor)

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
                                                    # num_words=top_k,
                                                    oov_token="<unk>",
                                                    filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

    tokenizer.fit_on_texts(df['caption'].tolist())
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'

    train_seqs = tokenizer.texts_to_sequences(df['caption'].tolist())
    cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
    # max_length = calc_max_length(train_seqs)
    max_length = 23

    return tokenizer, cap_vector, max_length

tokenizer, cap_vector, max_length = get_tokenizer(train_df)

In [ ]:
# Taille du vocabulaire
print(len(tokenizer.word_index))

#Liste triée dans l'ordre de la fréquence décroissante
print(sorted(list(tokenizer.word_counts.items()),key=lambda x: -x[1])[:20])

print(cap_vector[0]) 


In [ ]:
# reconstrcut sentence of cap_vector[0] using tokenizer
print(" ".join([tokenizer.index_word[i] for i in cap_vector[0] if i not in [0]]))

In [ ]:
embedding_dict = {}

f=open(os.path.join(base_path, 'glove', 'glove.6B.'+str(glove_dim)+'d.txt'),'r',encoding='utf-8')
for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:],'float32')
        # Transformer chaque mot en un vecteur de dimensions 100.
        embedding_dict[word]=vectors
        
f.close()


In [ ]:

# Matrice initiale emmbedding pour notre jeu de données
hit=0
misses={}

# Nombre de token ( numpy est à base zéro)
num_words = len(tokenizer.word_index) + 1

# Dimension de représentation =100 selon Glove choisi
embedding_matrix = np.zeros((num_words, glove_dim))

# On remplit la matrice avec les coordonnées issues de la représentation pré-entrainée
# Pourvu que le terme de notre dictionnaire recherché soit présent dans la représentation pré-entrainée de GloVe
for word, i in tokenizer.word_index.items():
    if i > num_words:
        continue
    
    emb_vec = embedding_dict.get(word)
    
    if emb_vec is not None:
        embedding_matrix[i] = emb_vec
        # print(embedding_matrix[i])
        hit = hit+1
    else:
        if word not in misses.keys():
            misses[word] = 1
        else:
            misses[word] += 1
        
# Affichage de controle: le nombre de termes trouvés et non trouvés dans la représentation pré-entrainée de GloVe.

print('Trouvés %s mots.' % hit)
print('Non trouvés %s mots.' % len(misses))

new_list = sorted(misses, reverse=True)

In [ ]:
# show top most 10 words not found in glove from the object misses created above
print(sorted(misses.items(), reverse=True))

# Prep Data Image

In [ ]:
# Telechargement du modèle InceptionV3 pré-entrainé avec la cassification sur ImageNet
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
for layer in image_model.layers:
    layer.trainable = False
# Creation d'une variable qui sera l'entrée du nouveau modèle de pre-traitement d'images
new_input = image_model.input
# récupérer la dernière couche caché qui contient l'image en representation compacte
hidden_layer = image_model.layers[-1].output


# Modèle qui calcule une representation dense des images avec InceptionV3
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

# Définition de la fonction load_image
def load_image(image_path):
    """
    La fonction load_image a pour entrée le chemin d'une image et pour sortie un couple
    contenant l'image traitée ainsi que son chemin dtop_k = 5000accès.
    La fonction load_image effectue les traitement suivant:
        1. Chargement du fichier correspondant au chemin d'accès image_path
        2. Décodage de l'image en RGB.
        3. Redimensionnement de l'image en taille (299, 299).
        4. Normalisation des pîxels de l'image entre -1 et 1
    """
    img = tf.io.read_file(image_path)  # 1. Chargement du fichier
    img = tf.image.decode_jpeg(img, channels=3)  # 2. Décodage en RGB
    img = tf.image.resize(img, (299, 299))  # 3. Redimensionnement à 299x299
    img = tf.keras.applications.inception_v3.preprocess_input(img)  # 4. Normalisation entre -1 et 1
    return img, image_path 

# Pré-traitement des images
# Prendre les noms des images
encode_train = sorted(set(train_df['image_path']))

# Creation d'une instance de "tf.data.Dataset" partant des noms des images 
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
# Division du données en batchs après application du pré-traitement fait par load_image
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

# Parcourir le dataset batch par batch pour effectuez le pré-traitement d'InceptionV3



df = pd.read_pickle('/home/cesi/datascience/data/captions_train2014_souf.pkl')
# def change_path(path):
#     return path.replace('/tf', base_path)
# df['image_path'] = df['image_path'].apply(lambda x : change_path(x))
# df.to_pickle('/home/cesi/datascience/data/captions_train2014_souf.pkl')

# for img, path in tqdm(image_dataset):
#     # Pré-traitement du batch (de taille (16,8,8,2048)) courant par InceptionV3 
#     batch_features = image_features_extract_model(img)
#     # Resize du batch de taille (16,8,8,2048) en taille (16,64,2048)
#     batch_features = tf.reshape(batch_features,
#                               (batch_features.shape[0], -1, batch_features.shape[3]))
#     # Parcourir le batch courant et stocker le chemin ainsi que le batch avec np.save()
#     for bf, p in zip(batch_features, path):
#         path_of_feature = p.numpy().decode("utf-8")
#         # (chemin de l'image associe a sa nouvelle representation , representation de l'image)
#         np.save(path_of_feature, bf.numpy())


# Dataset

In [ ]:
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8')+'.npy')
    return img_tensor, cap


dataset = tf.data.Dataset.from_tensor_slices((train_df['image_path'].values, cap_vector ))
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Melanger les donnees et les diviser en batchs
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
# list(dataset.as_numpy_iterator())[0]

In [ ]:
list(dataset.take(1).as_numpy_iterator())[0][0].shape

# Encodeur
CNN + dense


In [ ]:
class CNN_Encoder(tf.keras.Model):
    # Comme les images sont déjà prétraités par InceptionV3 est représenté sous forme compacte
    # L'encodeur CNN ne fera que transmettre ces caractéristiques à une couche dense
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # forme après fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

# Attention


In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)  # Transformation pour features
        self.W2 = tf.keras.layers.Dense(units)  # Transformation pour hidden
        self.V = tf.keras.layers.Dense(1)  # Transformation pour score

    def call(self, features, hidden):
        # Ajout d'une dimension de temps à hidden pour le calcul
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                             self.W2(hidden_with_time_axis)))
        
        # Calcul du score
        score = self.V(attention_hidden_layer)

        # Poids d'attention
        attention_weights = tf.nn.softmax(score, axis=1)

        # Vecteur de contexte
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        #context_vector est 

        return context_vector, attention_weights


# Decoder

In [ ]:

class RNN_Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(
                                        len(tokenizer.word_index)+1,
                                        embedding_dim,
                                        embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
                                        trainable=False
                                    )


        self.gru = tf.keras.layers.GRU(self.units,  
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        # print("input decoder:",x.shape)
        # print(x)
    
        context_vector, attention_weights = self.attention(features, hidden)
        # print("context_vector.shape = ", context_vector.shape)
        # print("attention_weights.shape = ", attention_weights.shape)
        # Passage du mot courant à la couche embedding
        x = self.embedding(x)
        
        # print("post embedding:",x.shape)
        # print(x)
        # Concaténation du vecteur de contexte et du mot courant
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # print("post concat:",x.shape)
        # print(x)

        # Passage du vecteur concaténé à la gru
        output, state = self.gru(x)
        
        # print("post gru:",output.shape)
        # print(output)
        # print("post gru state:",state.shape)
        # print(state)

        # Couche dense intermédiaire
        y = self.fc1(output)
        
        y = tf.reshape(y, (-1, y.shape[2]))
        
        
        # Dernière couche dense pour prédire le prochain mot du vocabulaire
        y = self.fc2(y)

        return y, state, attention_weights

    def reset_state(self, batch_size):
        # print("reset_state:batch_size = ", batch_size)
        return tf.zeros((batch_size, self.units))


# Model

In [ ]:
print(len(tokenizer.word_index))

In [ ]:
encoder = CNN_Encoder(embedding_dim)
# Création du décodeur
decoder = RNN_Decoder(units, len(tokenizer.word_index))


# Optimiseur ADAM
optimizer = tf.keras.optimizers.Adam()

# La fonction de perte
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
# Checkpoint todo
# checkpoint_path = "./checkpoints/train"
# ckpt = tf.train.Checkpoint(encoder=encoder,
#                            decoder=decoder,
#                            optimizer = optimizer)
# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)


# start_epoch = 0
# if ckpt_manager.latest_checkpoint:
#     start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
#     # Restaurer le dernier checkpoint dans checkpoint_path
#     ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
#debug

for (batch, (img_tensor, target)) in enumerate(dataset):
    # print(target.shape[0])
    # print(target[0])
    # print([tokenizer.word_index['<start>']] * target.shape[0])
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    # print(dec_input[0])
    # print(img_tensor.shape)
    features = encoder(img_tensor)
    # print(features.shape)


   
    predictions, hidden, _ = decoder(dec_input, features, hidden)

    print(predictions.shape)
    print(target.shape)

    break
    # print([tokenizer.word_index['<start>']] *  target.shape[0])

In [ ]:
int(dataset.__len__())

In [ ]:
loss_plot = []
@tf.function
def train_step(img_tensor, target):
    loss = 0

    # Initialisation de l'état caché pour chaque batch
    hidden = decoder.reset_state(batch_size=target.shape[0])
    
    # Initialiser l'entrée du décodeur
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    
    

    with tf.GradientTape() as tape: # Offre la possibilité de calculer le gradient du loss
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # Prédiction des i'èmes mot du batch avec le décodeur
            predictions, hidden, _ = decoder(dec_input, features, hidden)
            loss += loss_function(target[:, i], predictions)

            # Le mot correct à l'étap i est donné en entrée à l'étape (i+1)
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

EPOCHS = 10
# val_dataset = dataset[:len(dataset)//20]
# dataset = dataset[len(dataset)//20:]

for epoch in range(0, EPOCHS): # start_epoch, EPCHOS
    start = time.time()
    total_loss = 0
    
    for (batch, (img_tensor, target)) in tqdm(enumerate(dataset)):
        # print("batch = ", batch)
        # print("img_tensor = ", img_tensor)
        # print("target = ", target)
        
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
            # break
        
    
    # sauvegarde de la perte
    loss_plot.append(total_loss / int(dataset.__len__()))
    
    # if epoch % 5 == 0:
    #     ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/int(dataset.__len__())))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# Affichage de la courbe d'entrainement
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# Affichage de la courbe d'entrainement
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

In [ ]:
attention_features_shape = 64

def evaluate_loss(image, label):
    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            break

        dec_input = tf.expand_dims([predicted_id], 0)

    loss = loss_function(label[:, i], predictions)

    return loss


def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

import math
# Fonction permettant la représentation de l'attention au niveau de l'image
# def plot_attention(image, result, attention_plot):
#     temp_image = np.array(Image.open(image))

#     fig = plt.figure(figsize=(10, 10))

#     len_result = len(result)
#     for l in range(len_result):
#         temp_att = np.resize(attention_plot[l], (8, 8))
#         ax = fig.add_subplot(len_result//2, len_result//2, l+1)
#         ax.set_title(result[l])
#         img = ax.imshow(temp_image)
#         ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

#     plt.tight_layout()
#     plt.show()

def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))
    fig = plt.figure(figsize=(10, 10))
    
    len_result = len(result)
    grid_size = math.ceil(math.sqrt(len_result))
    
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(grid_size, grid_size, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())
    plt.savefig("/home/cesi/datascience/project/Livrable_3/attention.png")
    plt.tight_layout()
    plt.show()

# Affichage de quelques annotations dans le jeu de test
# rid = np.random.randint(0, len(img_name_val))

rid = 10

# image = img_name_val[rid]
image = df.iloc[rid]['image_path']

# real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
real_caption = df.iloc[rid]['caption']

result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)